## ETL Run Notebook  - Run scripts & Test w/ sql queries

### Run create_table.py to see if tables can be created property

- Make sure you open a RedShift cluster and make it publicly accessible

In [8]:
 pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [32]:
import psycopg2 

try:
    %run create_table.py 
except Exception as e:
    print(e)

CREATE TABLE IF NOT EXISTS surfacetemp(
        surfacetemp_id SERIAL PRIMARY KEY NOT NULL,
        year int,
        month int,
        country varchar,
        avg_temperature double, 
        avg_temperature_uncertainty double
        );
        


FeatureNotSupported: Column "surfacetemp.surfacetemp_id" has unsupported type "serial".


In [22]:
try:
    %run etl.py 
except Exception as e:
    print(e)

OperationalError: could not translate host name "dwhcluster.cslgr7lpwfo1.us-west-2.redshift.amazonaws.com" to address: Unknown host


#### Test tables with some Queries